In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import swat
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
os.environ["CAS_CLIENT_SSL_CA_LIST"]="/home/ds/cascert.pem"

conn = swat.CAS("d-crd-datasci01big.dev.cyber.sas.com", 5570)

In [3]:
conn.setsessopt('WH')

+ Elapsed: 0.000361s, user: 0.000114s, sys: 0.000206s, mem: 0.225mb

In [4]:
out = conn.loadtable('wls_day_02_hr13.sashdat', caslib='WH')

NOTE: Cloud Analytic Services made the file wls_day_02_hr13.sashdat available as table WLS_DAY_02_HR13 in caslib CASUSER(daherr).


In [5]:
conn.fetch(table={'name':'wls_day_02_hr13'}, maxrows=20)

,AuthenticationPackage,Destination,DomainName,EventID,FailureReason,LogHost,LogonID,LogonType,LogonTypeDescription,ParentProcessID,...,ProcessName,ServiceName,Source,Status,SubjectDomainName,SubjectLogonID,SubjectUserName,Time,UserName,occurredTime
0,,,Domain001,4688.0,,Comp716863,0x3e7,NaN,,0x238,...,conhost.exe,,,,,,,133201.0,Comp716863$,13:00:01
1,,,Domain001,4688.0,,Comp558334,0x3e7,NaN,,0x3a4,...,dllhost.exe,,,,,,,133201.0,Comp558334$,13:00:01
2,,,Domain001,4672.0,,Comp571028,0xf435131,NaN,,,...,,,,,,,,133201.0,Administrator,13:00:01
3,,,Domain001,4672.0,,Comp571028,0xf43667e,NaN,,,...,,,,,,,,133201.0,Administrator,13:00:01
4,,,Domain001,4688.0,,Comp330510,0x3e7,NaN,,0x1694,...,Proc915458.exe,,,,,,,133201.0,Comp330510$,13:00:01
5,,,Domain001,4688.0,,Comp620425,0x3e7,NaN,,0x898,...,cscript.exe,,,,,,,133201.0,Comp620425$,13:00:01
6,,,Domain001,4688.0,,Comp239408,0x3e7,NaN,,0x221c,...,conhost.exe,,,,,,,133201.0,Comp239408$,13:00:01
7,,,Domain001,4634.0,,Comp507073,0xa8a8e23d,7.0,Unlock,,...,,,,,,,,133201.0,User325570,13:00:01
8,,,Domain001,4688.0,,Comp640974,0x3e7,NaN,,0x19b0,...,conhost.exe,,,,,,,133201.0,Comp640974$,13:00:01
9,,,Domain001,4688.0,,Comp288986,0x3e7,NaN,,0x1ec,...,conhost.exe,,,,,,,133201.0,Comp288986$,13:00:01


In [36]:
keepCols = conn.fetch(table={'name':'wls_day_02_hr13'}, maxrows=20)['Fetch'].keys().values.tolist()
keepColsStr = " ".join(keepCols)

sasCode = F'''
    data wls_day_02_hr13_pg;
        set wls_day_02_hr13;
        if find(UserName, '$', 'i') then peerGroup = "Computers";
        else if find(UserName, 'User') then peerGroup = "Users";
        else peerGroup = "Computers";
'''

print(sasCode)


    data wls_day_02_hr13_pg;
        set wls_day_02_hr13;
        if find(UserName, '$', 'i') then peerGroup = "Computers";
        else if find(UserName, 'User') then peerGroup = "Users";
        else peerGroup = "Computers";



In [37]:
if conn.table.tableexists(name='wls_day_02_hr13_pg'):
    conn.table.droptable(name='wls_day_02_hr13_pg')

conn.datastep.runcode(sasCode)
conn.promote(name='wls_day_02_hr13_pg')

NOTE: Cloud Analytic Services dropped table wls_day_02_hr13_pg from caslib CASUSER(daherr).


+ Elapsed: 0.0873s, user: 0.00688s, sys: 0.0798s, mem: 0.272mb

In [38]:
conn.caslibinfo(caslib='WH')

,Name,Type,Description,Path,Definition,Subdirs,Local,Active,Personal,Hidden,Transient
0,WH,PATH,Windows Host Events,/home/datasets/LANL/WH/,,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
conn.fetch(table={'name':'wls_day_02_hr13_pg'}, maxrows=20)

,AuthenticationPackage,Destination,DomainName,EventID,FailureReason,LogHost,LogonID,LogonType,LogonTypeDescription,ParentProcessID,...,ServiceName,Source,Status,SubjectDomainName,SubjectLogonID,SubjectUserName,Time,UserName,occurredTime,peerGroup
0,Kerberos,,Domain001,4624.0,,ActiveDirectory,0x15b1ad12,3.0,Network,,...,,Comp382519,,,,,133668.0,User999475,13:07:48,Users
1,MICROSOFT_AUTHENTICATION_PACKAGE_V1_0,,Domain001,4776.0,,ActiveDirectory,,NaN,,,...,,Comp807801,0x0,,,,133668.0,AppService,13:07:48,Computers
2,,,Domain001,4769.0,,ActiveDirectory,,NaN,,,...,Comp916004$,Comp916004,0x0,,,,133668.0,User799415,13:07:48,Users
3,MICROSOFT_AUTHENTICATION_PACKAGE_V1_0,,Domain001,4776.0,,ActiveDirectory,,NaN,,,...,,Comp870313,0x0,,,,133668.0,User490190,13:07:48,Users
4,,,Domain001,4688.0,,Comp864731,0x3e7,NaN,,0x270,...,,,,,,,133668.0,Comp864731$,13:07:48,Computers
5,,,Domain001,4688.0,,Comp897367,0x3e7,NaN,,0xf80,...,,,,,,,133668.0,Comp897367$,13:07:48,Computers
6,Kerberos,,Domain001,4624.0,,Comp256596,0x7c62ccf,3.0,Network,,...,,Comp809819,,,,,133668.0,Comp809819$,13:07:48,Computers
7,,,Domain001,4634.0,,Comp256596,0x7c5f65a,3.0,Network,,...,,,,,,,133668.0,Comp413062$,13:07:48,Computers
8,,,Domain001,4634.0,,Comp256596,0x7c5f4e9,3.0,Network,,...,,,,,,,133668.0,User088300,13:07:48,Users
9,,,Domain001,4688.0,,Comp714124,0x3e7,NaN,,0x390,...,,,,,,,133668.0,Comp714124$,13:07:48,Computers


In [42]:
conn.table.save(table='wls_day_02_hr13_pg', name='wls_day_02_hr13_pg.sashdat', caslib='WH', replace=True)

NOTE: Cloud Analytic Services saved the file wls_day_02_hr13_pg.sashdat in caslib WH.


[caslib]

 'WH'

[name]

 'wls_day_02_hr13_pg.sashdat'

+ Elapsed: 0.422s, user: 0.0612s, sys: 0.336s, mem: 16.4mb

In [43]:
conn.close()